In [ ]:
I_GPU = 0

In [ ]:
# %load_ext autoreload
# %autoreload 2
import os
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
from torch_points3d.datasets.segmentation.multimodal.s3dis import S3DISFusedDataset
from torch_points3d.datasets.segmentation.multimodal import IGNORE_LABEL

_ = torch.cuda.is_available()
_ = torch.cuda.memory_allocated()
torch.cuda.set_device(I_GPU)

# Load config

In [ ]:
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read

# Set root to the DATA drive, where the data was downloaded
# DATA_ROOT = '/mnt/fa444ffd-fdb4-4701-88e7-f00297a8e29b/projects/datasets/s3dis'  # ???
# DATA_ROOT = '/media/drobert-admin/DATA/datasets/s3dis'  # IGN DATA
DATA_ROOT = '/media/drobert-admin/DATA2/datasets/s3dis'  # IGN DATA2
# DATA_ROOT = '/var/data/drobert/datasets/s3dis'  # AI4GEO
# DATA_ROOT = '/home/qt/robertda/scratch/datasets/s3dis'  # CNES
# DATA_ROOT = '/raid/datasets/pointcloud/data/s3dis'  # ENGIE

overrides = [
    'task=segmentation',
    'data=segmentation/multimodal/s3disfused/3d_2d/sparse/no_pixel_height/5cm_1024x512-exact',
    'models=segmentation/multimodal/no3d',
    'model_name=XYZ-RGB-L4-late-QKV',
    'data.fold=5',
    'data.sample_per_epoch=2000',
    f"data.dataroot={os.path.join(DATA_ROOT, '5cm_exact_1024x512')}",
]

cfg = hydra_read(overrides)

# print(OmegaConf.to_yaml(cfg))# Load S3DIS dataset

# Load dataset

In [ ]:
CLASSES = [
    'ceiling',
    'floor',
    'wall',
    'beam',
    'column',
    'window',
    'door',
    'chair',
    'table',
    'bookcase',
    'sofa',
    'board',
    'clutter',
]
OBJECT_COLOR = [
    [180, 180, 80],  #'ceiling' .-> .yellow
    [95, 156, 196],  #'floor' .-> . blue
    [179, 116, 81],  #'wall'  ->  brown
    [241, 149, 131],  #'beam'  ->  salmon
    [81, 163, 148],  #'column'  ->  bluegreen
    [77, 174, 84],  #'window'  ->  bright green
    [108, 135, 75],  #'door'   ->  dark green
    [41, 49, 101],  #'chair'  ->  darkblue
    [79, 79, 76],  #'table'  ->  dark grey
    [223, 52, 52],  #'bookcase'  ->  red
    [89, 47, 95],  #'sofa'  ->  purple
    [81, 109, 114],  #'board'   ->  grey
    [125, 125, 125],  #'clutter'  ->  light grey
]

num_classes = len(CLASSES)

In [ ]:
from time import time
start = time()
    
dataset = S3DISFusedDataset(cfg.data)
# print(dataset)

# Remove pixel memory credit transform
from torch_points3d.core.data_transform.multimodal.image import PickImagesFromMemoryCredit
from torch_points3d.datasets.base_dataset_multimodal import BaseDatasetMM
# dataset.test_dataset[0].transform_image = BaseDatasetMM.remove_multimodal_transform(dataset.test_dataset[0].transform_image, [PickImagesFromMemoryCredit])

print(f"Time = {time() - start:0.1f} sec.")

# Load model

### Instantiate model

In [ ]:
# from torch_points3d.models.model_factory import instantiate_model
# 
# model = instantiate_model(cfg, dataset)
# model = model.train().cuda()

### Load model from checkpoint

In [ ]:
from torch_points3d.metrics.model_checkpoint import ModelCheckpoint

# checkpoint_dir = '/home/ign.fr/drobert-admin/Bureau/benchmark_checkpoints/benchmark-Res16UNet21-15_ResImage3_light_1_a4_concatenation-20210304_210217'
# checkpoint_dir = '/home/ign.fr/drobert-admin/Bureau/benchmark_checkpoints/benchmark-Res16UNet21-15_ResImage3_light_1_mean_concatenation-20210301_230608'
# checkpoint_dir = '/home/ign.fr/drobert-admin/Bureau/benchmark_checkpoints/XYZ+RGB_a4-dim_cat-1'
# checkpoint_dir = '/home/ign.fr/drobert-admin/Bureau/benchmark_checkpoints/XYZ+RGB_mean_cat-1'
# checkpoint_dir = '/home/ign.fr/drobert-admin/Bureau/benchmark_checkpoints/XYZ+RGB_attention_debug_fast/files'
# checkpoint_dir = '/workspace/projects/torch-points3d/outputs/benchmark/benchmark-Res16UNet21-15_light-20210330_193749/wandb/run-20210330_193750-1ltttctz/files'
# checkpoint_dir = '/home/ign.fr/drobert-admin/Bureau/benchmark_checkpoints/RGB_light_drop-50_view-loss_fold5'
checkpoint_dir = '/home/ign.fr/drobert-admin/projects/torch-points3d/outputs/benchmark/cnes/XYZ+RGB/XYZ-RGB-L4-late-QKV'

# RGB light drop 50 trained on "5cm exact 512x256"
# checkpoint_dir = '/home/drobert/projects/torch-points3d/outputs/benchmark/benchmark-Res16UNet21-15_light_drop-50_image-view-loss-20210419_193538'

# RGB light drop 50 trained on "5cm exact 768x384"
# checkpoint_dir = '/home/drobert/projects/torch-points3d/outputs/benchmark/benchmark-Res16UNet21-15_light_drop-50_image-view-loss-20210428_165425'
# checkpoint_dir = '/home/ign.fr/drobert-admin/projects/torch-points3d/outputs/benchmark/cnes/RGB/fold5/RGB_ResNet18PPM_gp-8-32-32+mod-128-1_gscale_LR-10-2-3-4_exact_1024x512'

# Load model from checkpoint
selection_stage = 'val' # train, val, test
weight_name = 'loss_seg'  # miou, macc, acc, ..., latest
checkpoint = ModelCheckpoint(checkpoint_dir, cfg.model_name, selection_stage, run_config=cfg, resume=False)
model = checkpoint.create_model(dataset, weight_name=weight_name)
model = model.eval().cuda()

In [ ]:
# Activate the save_last option to investigate QKVBimodalCSRPool module
i_pool_branch = 0
# i_pool_branch = 1
# model.backbone.down_modules[i_pool_branch].image.view_pool.save_last = True
model.backbone_no3d.down_modules[i_pool_branch].image.view_pool.save_last = True

# Inference

In [ ]:
from torch_points3d.core.multimodal.data import MMBatch
from torch_points3d.metrics.confusion_matrix import ConfusionMatrix
from tqdm import tqdm
from torch_points3d.modules.multimodal.pooling import BimodalCSRPool, QKVBimodalCSRPool, HeuristicBimodalCSRPool, GroupBimodalCSRPool

def inference(model, dataset, set_name='TEST', n_infer=1000):
    if set_name.upper() == 'TRAIN':
        dataset_ = dataset.train_dataset
    elif set_name.upper() == 'VAL':
        dataset_ = dataset.val_dataset
    elif set_name.upper() == 'TEST':
        dataset_ = dataset.test_dataset[0]
    else:
        raise ValueError(f"Unknown set '{set_name.upper()}'")

    c = ConfusionMatrix(dataset.num_classes)
    
#     attention = model.backbone.down_modules[i_pool_branch].image.view_pool
    attention = model.backbone_no3d.down_modules[i_pool_branch].image.view_pool
    
    idx = []
    group_size = []
    x_map = []
    x_mod = []
    if isinstance(attention, QKVBimodalCSRPool):
        K = []
        Q = []
    C = []
    A = []
    G = []
    Y = []
    Y_pred = []
    count = 0

    for i in tqdm(np.random.choice(len(dataset_), n_infer)):

        # Skip these two sphere samples, they make the model crash
        if i in [1470, 1471] and set_name.upper() == 'TEST':
            continue
        
        if i in [903] and set_name.upper() == 'TEST':
            continue

        batch = MMBatch.from_mm_data_list([dataset_[int(i)]])

        # ------------------------------------------------------------------
    #     # TEMPORARY FIX TO DROP SOME LOCAL PROJECTION FEATURES
    #     for s in range(batch.modalities['image'].num_settings):
    #         batch.modalities['image'][s].mappings.features = batch.modalities['image'][s].mappings.features[:, :-2]
        # ------------------------------------------------------------------

        if not model.is_multimodal:
            batch = batch.data

        model.set_input(batch, model.device)
        _ = model(batch)

        gt = model.labels.cpu().numpy()
        pred = model.output.argmax(dim=1).cpu().numpy()
        c.count_predicted_batch(gt, pred)

        # ------------------------------------------------------------------
        # POOLING DATA
        idx.append(attention._last_idx.detach().cpu() + count)
        group_size.append(attention._last_view_num.detach().cpu())
        x_map.append(attention._last_x_map.detach().cpu())
        x_mod.append(attention._last_x_mod.detach().cpu())
        if isinstance(attention, QKVBimodalCSRPool):
            K.append(attention._last_K.detach().cpu())
            Q.append(attention._last_Q.detach().cpu())
        C.append(attention._last_C.detach().cpu())
        A.append(attention._last_A.detach().cpu())
        G.append(attention._last_G.detach().cpu())
        Y.append(gt)
        Y_pred.append(pred)
        count += attention._last_idx.detach().cpu().max() + 1
        # ------------------------------------------------------------------


    oa = np.round(c.get_overall_accuracy() * 100, decimals=2)
    macc = np.round(c.get_mean_class_accuracy() * 100, decimals=2)
    miou = np.round(c.get_average_intersection_union() * 100, decimals=2)
    iou_dict = {dataset.INV_OBJECT_LABEL[k]: np.round(v * 100, decimals=1) 
                for k, v in enumerate(c.get_intersection_union_per_class()[0])}

    print(f"OA : {oa}")
    print(f"macc : {macc}")
    print(f"mIoU : {miou}")
    print("Per class IoU")
    for k, v in iou_dict.items():
        print(f"    {k:<9}: {v}")
    print()
    
    idx = torch.cat(idx, dim=0)
    group_size = torch.cat(group_size, dim=0)
    x_map = torch.cat(x_map, dim=0)
    x_mod = torch.cat(x_mod, dim=0)
    if isinstance(attention, QKVBimodalCSRPool):
        K = torch.cat(K, dim=0)
        Q = torch.cat(Q, dim=0)
    C = torch.cat(C, dim=0)
    A = torch.cat(A, dim=0)
    G = torch.cat(G, dim=0)
    
    Y = torch.cat([torch.from_numpy(y) for y in Y], dim=0)
    Y_pred = torch.cat([torch.from_numpy(y) for y in Y_pred], dim=0)
    
    group_size_view = torch.repeat_interleave(group_size, group_size)
    Y_view = torch.repeat_interleave(Y, group_size)
    Y_pred_view = torch.repeat_interleave(Y_pred, group_size)
    G_view = torch.repeat_interleave(G, group_size, dim=0)
    
    # For pure-2d mean logit fusion models, x_mod carries the logits
    Y_pred_view_indiv = torch.max(x_mod, dim=1).indices
    
    # Row-normalized confusion matrix - rows sum up to 1. Each row carries 
    # the distribution of predictions for an expected label. This 
    # illustrates how classes are misclassified.
    confusion = np.zeros((dataset.num_classes, dataset.num_classes))
    for i, j in zip(Y.numpy(), Y_pred.numpy()):
        confusion[i, j] += 1
    sns.heatmap(confusion / confusion.max(axis=1).reshape(-1, 1), xticklabels=CLASSES, yticklabels=CLASSES)
    plt.show()
    print()
    
    # Wrap up everything in an output dictionary
    out  = {
        'oa': oa,
        'macc': macc,
        'miou': miou,
        'iou_dict': iou_dict,
        'idx': idx,
        'group_size': group_size,
        'x_map': x_map,
        'x_mod': x_mod,
        'Y': Y,
        'Y_pred': Y_pred,
        'group_size_view': group_size_view,
        'Y_view': Y_view,
        'Y_pred_view': Y_pred_view,
        'Y_pred_view_indiv': Y_pred_view_indiv,
        'num_groups': attention.num_groups,
    }
    
    if isinstance(attention, QKVBimodalCSRPool):
        out['K'] = K
        out['Q'] = Q
    out['C'] = C
    out['A'] = A
    out['G'] = G
    out['G_view'] = G_view
    
    return out

In [ ]:
# Inference on TEST for average-pooled logit model
out = inference(model, dataset, set_name='TEST', n_infer=100)

In [ ]:
# Make 1D tensor 2D if need be
make_1d_2d = lambda x: x.unsqueeze(1) if x is not None and x.dim() < 2 else x

# Recover data
oa = out.get('oa', None)
macc = out.get('macc', None)
miou = out.get('miou', None)
iou_dict = out.get('iou_dict', None)
idx = make_1d_2d(out.get('idx', None))
group_size = out.get('group_size', None)
x_proj = out.get('x_proj', None)
x_mod = out.get('x_mod', None)
Y = out.get('Y', None)
Y_pred = out.get('Y_pred', None)
group_size_view = out.get('group_size_view', None)
Y_view = out.get('Y_view', None)
Y_pred_view = out.get('Y_pred_view', None)
Y_pred_view_indiv = out.get('Y_pred_view_indiv', None)
K = make_1d_2d(out.get('K', None))
Q = make_1d_2d(out.get('Q', None))
C = make_1d_2d(out.get('C', None))
A = make_1d_2d(out.get('A', None))
G = make_1d_2d(out.get('G', None))
G_view = make_1d_2d(out.get('G_view', None))
num_groups = out.get('num_groups', None)

# idx_compact values hold view indices so that all elements are seen 
img_idx = torch.unique(idx)
if img_idx.shape[0] < idx.max() + 1:
    idx_gen = torch.full((img_idx.max() + 1,), -1).long()
    idx_gen = idx_gen.scatter_(0, img_idx, torch.arange(img_idx.shape[0]))
    idx_compact = idx_gen[idx]
else:
    idx_compact = idx

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import torch_scatter

# N_PRINT = 500
N_PRINT = 50000

fig = plt.figure(figsize=(32, 6))
(ax0, ax1, ax2) = fig.subplots(1, 3)
palette = sns.color_palette("muted")
hist_kws = {'element': 'step', 'stat': 'probability', 'kde': True}

N_per_index = torch_scatter.scatter(torch.ones_like(A[:N_PRINT, 0]), idx_compact[:N_PRINT, 0], reduce='sum')
N_per_src_element = N_per_index.gather(0, idx_compact[:N_PRINT, 0])
H2 = torch_scatter.scatter((-A[:N_PRINT] * torch.log(A[:N_PRINT])), idx_compact[:N_PRINT], reduce='sum', dim=0) / N_per_index.log().unsqueeze(1)
H2[torch.where(H2.isinf())] = 0  # for edge cases

for i_group in range(num_groups):  
    sns.histplot(H2[:, i_group].numpy(), color=palette[i_group], label=f'Normalized entropy group={i_group}', ax=ax0, **hist_kws)

for i_group in range(num_groups):
    sns.histplot((A[:N_PRINT, i_group] * N_per_src_element).numpy(), color=palette[i_group], label=f'A*N group={i_group}', ax=ax1, **hist_kws)

for i_group in range(num_groups):
    sns.histplot(A[:N_PRINT, i_group].numpy(), color=palette[i_group], label=f'A group={i_group}', ax=ax2, **hist_kws)

for ax in (ax0, ax1, ax2):
    ax.legend()
    ax.set_ylabel('')
plt.show()

print()
print()

Y_print = torch_scatter.scatter(Y_view[:N_PRINT], idx_compact[:N_PRINT, 0], reduce='mean').long()

palette = sns.color_palette("muted")
hist_kws = {'element': 'step', 'stat': 'probability', 'kde': True}
    
for i_c, c in enumerate(Y_print.unique()):
    
    idx_class = torch.where(Y_print == c)[0]
    
    fig = plt.figure(figsize=(32, 6))
    fig.suptitle(f"Attention for '{CLASSES[c].upper()}' - N={idx_class.shape[0]}")
    (ax0, ax1, ax2) = fig.subplots(1, 3)
    
    for i_group in range(num_groups):  
        sns.histplot(H2[:, i_group][idx_class].numpy(), color=palette[i_group], label=f'Normalied entropy group={i_group}', ax=ax0, **hist_kws)

    for i_group in range(num_groups):
        sns.histplot((A[:N_PRINT, i_group] * N_per_src_element)[idx_class].numpy(), color=palette[i_group], label=f'A*N group={i_group}', ax=ax1, **hist_kws)

    for i_group in range(num_groups):
        sns.histplot(A[:N_PRINT, i_group][idx_class].numpy(), color=palette[i_group], label=f'A group={i_group}', ax=ax2, **hist_kws)

    for ax in (ax0, ax1, ax2):
        ax.legend()
        ax.set_ylabel('')
    plt.show()
    print()

In [ ]:
N_PRINT = 50000

fig = plt.figure(figsize=(10, 4 * 7))
(ax0, ax1, ax2, ax3, ax4, ax5, ax6) = fig.subplots(7, 1)

palette = sns.color_palette("muted")
hist_kws = {'element': 'step', 'stat': 'probability', 'kde': True}

# Multi-view bag size
sns.histplot(group_size.numpy(), bins=np.arange(21), ax=ax0, element='step', stat='probability', kde=False)
ax0.axvline(x=group_size.numpy().mean(), ls=':', c='black')
ax0.title.set_text('Bag size')
    
for i_group in range(num_groups):
    
    # Compatibilities
    sns.histplot(C[:N_PRINT, i_group].clamp(-1, 1.5).numpy(), color=palette[i_group], label=f'group={i_group}', ax=ax1, **hist_kws)
    ax1.title.set_text('Compatibility')
    
    # Attention
    sns.histplot(A[:N_PRINT, i_group].clamp(-1, 1.5).numpy(), color=palette[i_group], label=f'group={i_group}', ax=ax2, **hist_kws)
    for k in range(1, 10):
        ax2.axvline(x=1/k, ls=':', c='black')
    ax2.title.set_text('Attention')
    
    # A * bag size
    AN = A * torch.repeat_interleave(group_size, group_size, dim=0).unsqueeze(1)
    sns.histplot(AN[:N_PRINT, i_group].numpy(), color=palette[i_group], label=f'group={i_group}', ax=ax3, **hist_kws)
    ax3.title.set_text('Attention * Bag size')

    # Gating
    sns.histplot(G[:, i_group].clamp(-1, 1.5).numpy(), color=palette[i_group], label=f'group={i_group}', ax=ax4, **hist_kws)
    ax4.title.set_text('Gating')
    
# Y
sns.histplot(Y.numpy(), stat='probability', ax=ax5)
ax5.title.set_text('Y')

# Y_pred
sns.histplot(Y_pred.numpy(), stat='probability', ax=ax6)
ax6.title.set_text('Y_pred')

for ax in (ax0, ax1, ax2, ax3, ax4, ax5, ax6):
    ax.legend()
    ax.set_ylabel('')
plt.show()

In [ ]:
fig = plt.figure(figsize=(32, 2))
axes = fig.subplots(1, num_classes)
for i_class, ax in enumerate(axes):
    idx_class = torch.where(Y == i_class)
    group_size_class = group_size[idx_class]
    sns.distplot(
        torch.clamp(group_size_class, 0, 10).numpy()[:N_PRINT],
        kde_kws={'bw': 0.3}, 
        bins=np.arange(11),
        label=f"{CLASSES[i_class]}", 
        color=tuple([x / 255. for x in OBJECT_COLOR[i_class]]),
        ax=ax)
    ax.legend()
    ax.set_ylabel('')
    ax.axvline(x=group_size.numpy()[:N_PRINT].mean(), ls=':', c='black')
    ax.axvline(x=group_size_class.numpy()[:N_PRINT].mean(), ls=':', c='red')
plt.suptitle("Per-class group sizes")
plt.show()

In [ ]:
fig = plt.figure(figsize=(32, 4))
(ax0, ax1, ax2, ax3, ax4, ax5) = fig.subplots(1, 6)

sns.distplot(K[(x_map < 0.3).squeeze()].numpy()[:N_PRINT], label="K toxic", ax=ax0)
sns.distplot(K[(x_map >= 0.3).squeeze()].numpy()[:N_PRINT], label="K safe", ax=ax0)

sns.distplot(Q[(x_map < 0.3).squeeze()].numpy()[:N_PRINT], label="Q toxic", ax=ax1)
sns.distplot(Q[(x_map >= 0.3).squeeze()].numpy()[:N_PRINT], label="Q safe", ax=ax1)

sns.distplot(C[(x_map < 0.3).squeeze()].numpy()[:N_PRINT], label="Compatibility toxic", ax=ax2)
sns.distplot(C[(x_map >= 0.3).squeeze()].numpy()[:N_PRINT], label="Compatibility safe", ax=ax2)

sns.distplot(A[(x_map < 0.3).squeeze()].numpy()[:N_PRINT], label="Attention toxic", ax=ax3)
sns.distplot(A[(x_map >= 0.3).squeeze()].numpy()[:N_PRINT], label="Attention safe", ax=ax3)

for k in range(2, 10):
    idx_group = group_size_view == k
    sns.distplot(A[idx_group][(x_map[idx_group] < 0.3).squeeze()].numpy()[:N_PRINT], label=f"Attention toxic k={k}", ax=ax4)
    
for k in range(2, 10):
    idx_group = group_size_view == k
    sns.distplot(A[idx_group][(x_map[idx_group] >= 0.3).squeeze()].numpy()[:N_PRINT], label=f"Attention safe k={k}", ax=ax5)

for ax in (ax0, ax1, ax2, ax3, ax4, ax5):
    ax.legend()
    ax.set_ylabel('')
plt.show()

In [ ]:
model.eval()

batch = MMBatch.from_mm_data_list([dataset.test_dataset[0][int(i)]])
    
# ------------------------------------------------------------------
# TEMPORARY FIX TO DROP SOME LOCAL PROJECTION FEATURES
for s in range(batch.modalities['image'].num_settings):
    batch.modalities['image'][s].mappings.features = batch.modalities['image'][s].mappings.features[:, :-2]
# ------------------------------------------------------------------

if not model.is_multimodal:
    batch = batch.data

model.set_input(batch, model.device)
_ = model(batch)

gt = model.labels.cpu().numpy()
pred = model.output.argmax(dim=1).cpu().numpy()
c.count_predicted_batch(gt, pred)

In [ ]:
model.backward()

In [ ]:
attention.Q.weight.grad

In [ ]:
attention.Gating.bias

# Visualizing softmax properties

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

x = np.random.normal(loc=0.0, scale=1.0, size=10000)

sns.distplot(x)
sns.distplot(np.exp(x) * x)
plt.show()

print(f"mean x = {x.mean():0.2f}")
print(f"mean x*exp(x) = {(np.exp(x) * x).mean():0.2f}")

In [ ]:
n_items = 20
size = 10**4

x_sum = np.zeros(size)
for i in range(n_items):
    x = np.random.normal(loc=0.0, scale=1.0, size=size)
    x = x * np.exp(x)
    x_sum += x

x_sum = x_sum / n_items

sns.distplot(x_sum)
plt.show()
print(f"mean sum(x*exp(x)) = {x_sum.mean():0.2f}")

In [ ]:
x = np.arange(1, 30)
plt.plot(np.exp(-x), label="exp(-x)")
plt.plot(1/x, label="x^-1")
plt.plot(np.power(x, -0.5), label="x^-0.5")
plt.legend()
plt.show()